### Import required libraries

In [ ]:
import sounddevice as sd
import soundfile as sf
import liverecorder
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import time
from concurrent.futures import ThreadPoolExecutor
import threading
import math

### Identify our USB AD-converter (microphone)

In [ ]:
# Acceptable API:s in increasing order of preference
accept_apis = ['Windows WASAPI', 'Windows WDM-KS', 'ALSA', 'Core Audio']
all_apis = sd.query_hostapis()
useapi = -1
for pref in accept_apis:
    for idx, a in enumerate(sd.query_hostapis()):
        if a['name']==pref and len(a['devices'])>0:
            useapi = idx
if useapi<0:
    print('Will use any sound API')
else:
    print(f'Will use preferred sound API {all_apis[useapi]["name"]}')

In [ ]:
in_device  = sd.default.device['input']
in_devices = sd.query_devices()
for idx, d in enumerate(in_devices):
    if (useapi>=0 and d['hostapi']==useapi) and ('AudioStream' in d['name'] or 'LabDevice' in d['name']) and d['max_input_channels']>0:
        in_device = idx
print(f'Will use input device #{in_device} {in_devices[in_device]["name"]} with API {all_apis[in_devices[in_device]["hostapi"]]["name"]}')

### Define basic global state

We are not embarassed by this in a notebook...

In [ ]:
# Sample rate in Hz set on ADC by a GUI widget
samplerate     = widgets.BoundedIntText(value = 48000, min = 1000, max = 96000, step=100, description = "Sample rate")

current_data   = np.zeros(5 * samplerate.value, dtype=np.int16)
current_length = current_data.size

def time_to_samples(t):
    return round(t * samplerate.value)

window         = widgets.BoundedFloatText(value = 1.0, min = 0.001, max = current_data.size/samplerate.value, description = "Length")
offset         = widgets.BoundedFloatText(value = 0.0, min = 0.000, max = current_data.size/samplerate.value, description = "Offset")


### Define plots

In [ ]:
%matplotlib widget

ticks_milli = mpl.ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x * 1000))
ticks_micro = mpl.ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x * 1e6))

mpl.rcParams['path.simplify_threshold'] = 1.0

# Briefly turn interactive mode off to avoid making plots appear here, we will display them later
plt.ioff()
fig = plt.figure(figsize=(8, 8), label="ADC data analysis", constrained_layout=True);
plt.ion()

baseaxs, timeaxs, freqaxs = fig.subplots(3, 1, gridspec_kw = {'height_ratios': [1, 3,3]});

timetext = timeaxs.text(0.01, 0.99, 'Nothing yet...',
                        verticalalignment='top', horizontalalignment='left',
                        transform=timeaxs.transAxes,
                        color='orange', fontsize=15)

freqtext = freqaxs.text(0.01, 0.99, 'Nothing yet...',
                        verticalalignment='top', horizontalalignment='left',
                        transform=freqaxs.transAxes,
                        color='blue', fontsize=15)

baseaxs.set_ylim(-2048, 2048)
timeaxs.set_ylim(-2048, 2048)
freqaxs.set_ylim(-100, 10)

def get_time_points(rate, window, offset):
    return np.linspace((-window-offset)/rate, -offset/rate, window)

def get_freq_points(rate, window):
    return np.fft.rfftfreq(window, d=1./rate)

def get_time_ampl(data, window, offset):
    if offset != 0:
        return data[-window-offset:-offset]
    else:
        return data[-window:]

def get_freq_pwr(data, rate, window, offset):
    return 20 * np.log10(np.abs(np.fft.rfft(get_time_ampl(data, window, offset)) / window) + 1e-9) - 60

baseplt = baseaxs.plot([-1, 0], [0, 0], drawstyle = 'steps-post', color='orange')
timeplt = timeaxs.plot([0], [0], drawstyle = 'steps-post', color='orange')
freqplt = freqaxs.plot([0], [0], color='blue')

def window_select(min_x, max_x):
    window.value = round((max_x - min_x), 3)
    offset.value = round(-max_x, 3)

win_selector = mpl.widgets.SpanSelector(baseaxs, window_select, 'horizontal',
                                        interactive = True, drag_from_anywhere = True, ignore_event_outside = True)


def _update_spectrum(data, rate, window, offset):
    freq_pwr = get_freq_pwr(data, rate, window, offset)
    max_pwr_idx = np.argmax(freq_pwr)
    text = f'Peak {freq_pwr[max_pwr_idx]:.1f} dBV at {freqplt[0].get_xdata()[max_pwr_idx]:.1f} Hz'
    freqtext.set_text(text)
    freqplt[0].set_ydata(freq_pwr)
    fig.canvas.draw_idle()
    
def _update_baseeplot(data, rate):
    amplitudes = get_time_ampl(data, min(5*rate, data.size), 0)
    baseplt[0].set_ydata(amplitudes)
    fig.canvas.draw_idle()
    
def _update_timeplot(data, window, offset):
    amplitudes = get_time_ampl(data, window, offset)
    max_a = np.max(amplitudes)
    min_a = np.min(amplitudes)
    avg_a = np.mean(amplitudes)
    text = f'Max {max_a}, min {min_a}, avg {avg_a:.1f}'
    timetext.set_text(text)
    timeplt[0].set_ydata(amplitudes)
    fig.canvas.draw_idle()
    
def _update_rate(data, rate, win, off):
    fig.canvas.toolbar.home()
    length = data.size
    bwin = min(5*rate, length)
    btimes = get_time_points(rate, bwin, 0)
    baseplt[0].set_xdata(btimes)
    baseaxs.set_xlim(btimes[0], btimes[-1])
    win = min(win, length)
    window.value = min(win/rate, length/rate)
    window.max = length/rate
    off = min(off, length - win)
    offset.value = min(off/rate, (length - win)/rate)
    offset.max = (length - win)/rate
    win_selector.extents = (-window.value-offset.value, -offset.value)
    times = get_time_points(rate, win, off)
    timeplt[0].set_xdata(times)
    timeaxs.set_xlim(times[0], times[-1])
    freqs = get_freq_points(rate, win)
    freqplt[0].set_xdata(freqs)
    freqaxs.set_xlim(freqs[0], freqs[-1])
    fig.canvas.toolbar.update()
    
def update_data(data, rate, force = False):
    global current_data, current_length
    win = time_to_samples(window.value)
    off = time_to_samples(offset.value)
    if force or data.size != current_length:
        current_length = data.size
        _update_rate(data, rate, win, off)
    current_data = data
    _update_baseeplot(data, samplerate.value)
    _update_timeplot(data, win, off)
    _update_spectrum(data, samplerate.value, win, off)
    
update_data(current_data, samplerate.value, force = True)

### Capture samples from the ADC with live display

In [ ]:
adc_cnt = widgets.IntText(value = 0, description = "ADC samples", )
    
try:
    pool.shutdown()
except NameError:
    pass

pool = ThreadPoolExecutor(3)

def live_update(in_device, rate):
    
    def _update(rate):
        while liverecorder.running():
            liverecorder.get_update_event().wait(1)
            data, count = liverecorder.get_data()
            adc_cnt.value += count
            update_data(data, rate)
            time.sleep(0.2)
            
    liverecorder.start(in_device, rate)
    return pool.submit(_update, rate)    
        

### Define GUI

In [ ]:
def run_btn_name():
    return "Hold" if liverecorder.running() else "Capture"

run_button  = widgets.Button(description = run_btn_name())
play_button = widgets.Button(description = "Play data")

def toggle_capture(b):
    if liverecorder.running():
        liverecorder.stop()
    else:
        live_update(in_device, samplerate.value)
    b.description = run_btn_name()
    
def toggle_live(b):
    global is_live
    is_live = not is_live
    b.description = live_btn_name()
    
def is_playing():
    playing = False
    try:
        playing = sd.get_stream().active
    except RuntimeError:
        pass
    return playing
    
def play_data(b):
    if is_playing():
        sd.stop()
    else:
        sound = get_time_ampl(current_data, time_to_samples(window.value), time_to_samples(offset.value)).copy()
        sd.play(16*sound, samplerate=samplerate.value, device=sd.default.device['output'])
    
def rate_window_change(change):
    update_data(current_data, samplerate.value, force = True)
    
samplerate.observe(rate_window_change, names='value')
window.observe(rate_window_change, names='value')
offset.observe(rate_window_change, names='value')

run_button.on_click(toggle_capture)
play_button.on_click(play_data)

widgets.VBox([widgets.HBox([samplerate, adc_cnt]),
              widgets.HBox([run_button, play_button]),
              widgets.HBox([window, offset]),
              fig.canvas
             ])

In [ ]:
# np.save('vissling', data)


In [ ]:
#update_data(np.load('vissling.npy'), 24000, force = True)


In [ ]:
#current_length